In [1]:
import pandas as pd
import numpy as np
from causalnex.structure import StructureModel

In [2]:
from causalnex.plots import plot_structure
from causalnex.network import BayesianNetwork

/home/getachew/Documents/10_Academy/Week_8/Logistics_Optimization_with_Casual_Inference/causal_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from sklearn.model_selection import train_test_split
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE
import matplotlib.pyplot as plt
from causalnex.structure.notears import from_pandas
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [6]:
df = pd.read_csv('../data/features.csv')
df.head(3)

,id,order_id,driver_id,driver_action,lat,lng,driver_location,trip_origin,trip_destination,trip_Start_time,trip_end_time,trip_distance,trip_to_driver_distance,trip_time,trip_speed,isHoliday,isWeekend
0,1,392001,243828,accepted,6.602207,3.270465,"6.6022066,3.2704649","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,0.694264,0 days 00:03:37,0.096702,0,0
1,2,392001,243588,rejected,6.592097,3.287445,"6.5920972,3.2874447","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,1.551694,0 days 00:03:37,0.096702,0,0
2,3,392001,243830,rejected,6.596133,3.281784,"6.5961334,3.2817841","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,0.786777,0 days 00:03:37,0.096702,0,0


In [13]:
df["trip_time"] = pd.to_timedelta(df["trip_time"])
df["trip_time_seconds"] = df["trip_time"].dt.seconds
df["fulfilled"] = (df["driver_action"] == "accepted") & (df["trip_time_seconds"] >= 600)  # 600 seconds is equivalent to 10 minutes
df.head()

,id,order_id,driver_id,driver_action,lat,lng,driver_location,trip_origin,trip_destination,trip_Start_time,trip_end_time,trip_distance,trip_to_driver_distance,trip_time,trip_speed,isHoliday,isWeekend,trip_time_seconds,fulfilled
0,1,392001,243828,0,6.602207,3.270465,"6.6022066,3.2704649","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,0.694264,0 days 00:03:37,0.096702,0,0,217,False
1,2,392001,243588,1,6.592097,3.287445,"6.5920972,3.2874447","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,1.551694,0 days 00:03:37,0.096702,0,0,217,False
2,3,392001,243830,1,6.596133,3.281784,"6.5961334,3.2817841","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,0.786777,0 days 00:03:37,0.096702,0,0,217,False
3,4,392001,243539,1,6.596142,3.280526,"6.5961416,3.2805263","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,0.692054,0 days 00:03:37,0.096702,0,0,217,False
4,5,392001,171653,1,6.609232,3.288800,"6.6092317,3.2887999","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,20.984319,1.621848,0 days 00:03:37,0.096702,0,0,217,False


In [14]:
def labe_encoder(df:pd.DataFrame,columns:list=None):
    if columns == None:
        # columns = df.select_dtypes(exclude = ['number','datetime'])
        columns = df.select_dtypes(exclude = ['number'])
    le = LabelEncoder()

    for col in columns:
        df[col] = le.fit_transform(df[col])

    return df

In [15]:
# df = cleaner.label_encoder(df)
df = labe_encoder(df)

In [21]:
driver_loc=df[['driver_location']]
request_loc=df[["trip_origin"]]


In [22]:
# create kmeans model/object
from sklearn.cluster import KMeans
kmeans = KMeans(
    init="random",
    n_clusters=50,
    n_init=10,
    max_iter=300,
    random_state=42
)

In [19]:
df.head()

,id,order_id,driver_id,driver_action,lat,lng,driver_location,trip_origin,trip_destination,trip_Start_time,trip_end_time,trip_distance,trip_to_driver_distance,trip_time,trip_speed,isHoliday,isWeekend,trip_time_seconds,fulfilled
0,1,392001,243828,0,6.602207,3.270465,176330,11676,3785,275,123,20.984319,0.694264,0 days 00:03:37,0.096702,0,0,217,0
1,2,392001,243588,1,6.592097,3.287445,153354,11676,3785,275,123,20.984319,1.551694,0 days 00:03:37,0.096702,0,0,217,0
2,3,392001,243830,1,6.596133,3.281784,164981,11676,3785,275,123,20.984319,0.786777,0 days 00:03:37,0.096702,0,0,217,0
3,4,392001,243539,1,6.596142,3.280526,165009,11676,3785,275,123,20.984319,0.692054,0 days 00:03:37,0.096702,0,0,217,0
4,5,392001,171653,1,6.609232,3.288800,184461,11676,3785,275,123,20.984319,1.621848,0 days 00:03:37,0.096702,0,0,217,0


In [25]:
# do clustering
kmeans.fit(driver_loc)
# save results
driver_labels = kmeans.labels_
kmeans.fit(request_loc)
# save results
request_labels = kmeans.labels_

In [26]:
df["driver_loc"]=driver_labels
df["request_loc"]=request_labels

In [27]:
df.head()

,id,order_id,driver_id,driver_action,lat,lng,driver_location,trip_origin,trip_destination,trip_Start_time,...,trip_distance,trip_to_driver_distance,trip_time,trip_speed,isHoliday,isWeekend,trip_time_seconds,fulfilled,driver_loc,request_loc
0,1,392001,243828,0,6.602207,3.270465,176330,11676,3785,275,...,20.984319,0.694264,0 days 00:03:37,0.096702,0,0,217,0,48,37
1,2,392001,243588,1,6.592097,3.287445,153354,11676,3785,275,...,20.984319,1.551694,0 days 00:03:37,0.096702,0,0,217,0,8,37
2,3,392001,243830,1,6.596133,3.281784,164981,11676,3785,275,...,20.984319,0.786777,0 days 00:03:37,0.096702,0,0,217,0,25,37
3,4,392001,243539,1,6.596142,3.280526,165009,11676,3785,275,...,20.984319,0.692054,0 days 00:03:37,0.096702,0,0,217,0,25,37
4,5,392001,171653,1,6.609232,3.288800,184461,11676,3785,275,...,20.984319,1.621848,0 days 00:03:37,0.096702,0,0,217,0,45,37


In [31]:
cleaned_driver_loc=df[["driver_location"]]
cleaned_request_loc=df[["trip_origin"]]

In [32]:
# do clustering
kmeans.fit(cleaned_request_loc)
# save results
cleaned_request_labels = kmeans.labels_

# do clustering
kmeans.fit(cleaned_driver_loc)
# save results
cleaned_driver_labels = kmeans.labels_

In [33]:
clustered_df=df.copy()
clustered_df["driver_loc"]=cleaned_driver_labels
clustered_df["request_loc"]=cleaned_request_labels

In [35]:
clustered_df.drop(columns=["order_id","driver_location","trip_origin","driver_action"],inplace=True)

In [37]:
clustered_df["month"]=clustered_df["trip_Start_time"].apply(lambda x: pd.to_datetime(x).month)
clustered_df["day"]=clustered_df["trip_Start_time"].apply(lambda x: pd.to_datetime(x).day)
clustered_df["hour"]=clustered_df["trip_Start_time"].apply(lambda x: pd.to_datetime(x).hour)
clustered_df["minute"]=clustered_df["trip_Start_time"].apply(lambda x: pd.to_datetime(x).minute)
clustered_df.drop(columns="trip_Start_time",inplace=True)

In [40]:
clustered_df=labe_encoder(clustered_df,["isHoliday","driver_action","fulfilled","isWeekend"])

KeyError: 'driver_action'

In [41]:
no_edges=[("fulfilled","duration_min"),("fulfilled","driver_loc"),("driver_action","driver_loc"),("driver_action","distance_diff"),("driver_action","duration_min")]
sm_enhanced = from_pandas(clustered_df,tabu_child_nodes=["month","day","hour","minute"])

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''